In [190]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import re
import random
import uuid

In [176]:
chap = []
pars = []
refs = []
lines = []
for chapter in list(range(1,33)):
        
    url = f'https://1689londonbaptistconfession.com/{chapter}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    
    par = 0
    ref = 0
    divs = soup.find("div", {"class": "Container Chapter__chapter-container"}).find_all('div', attrs={'class': None, 'style': None})
    
    for div in divs:
        spans = div.find('p').find_all('span')        
        par += 1
        for i,span in enumerate(spans):
            if i % 2 != 0:
                continue    
            
            temp = span.find('span')
            noRef = False
            try:
                a = temp.find('a')
                a.extract()
                ref += 1
            except:
                noRef = True
            
            line = temp.text                        
            
            chap.append(chapter)
            pars.append(par)
            if noRef:
                refs.append(99)                        
            else:
                refs.append(ref)
            lines.append(line)    
        


In [177]:
original = pd.DataFrame({'chapter':chap, 'paragraph':pars, 'reference':refs,'line':lines})
original['scripture'] = 0


In [178]:
modern = pd.read_pickle('../data/1689_clean.pkl')

In [179]:
both = modern \
.merge(original, how='left', on=['chapter', 'paragraph', 'reference', 'scripture']) \
.rename(columns={'line_x':'modern', 'line_y':'original'})\
.fillna('')

In [180]:
both.to_excel('../data/temp_1689_both_version_to_fix.xlsx')

In [208]:
# get back from google drive
df = pd.read_excel('../data/temp_1689_both_version_to_fixed.xlsx')
df = df[['chapter','paragraph','reference','scripture','modern','original']]
df = df.rename(columns={'modern':'line'})
#patch
df['reference'] = df.apply(lambda x: 10 if x.line == 'Revelation 22:20' and x.chapter==32 else x.reference,axis=1 )


In [209]:
df.tail(1)

,chapter,paragraph,reference,scripture,line,original
1007,32,3,10,1,Revelation 22:20,NaN


In [210]:
df['reference'] = df.reference.fillna(99).astype(int)
df['line'] = df.line.apply(lambda x: x.replace('Psalm ', 'Psalms ') if 'Psalm ' in x else x)
df['line'] = df.line.apply(lambda x: x.replace('Psalms ', 'Psalm ') if 'Psalms ' in x else x)
df = df.sort_values(by=['scripture', 'chapter', 'paragraph', 'reference']).rename(columns={'reference':'referenceNum'})
text = df[df.scripture==0].drop(columns=['scripture',]).rename(columns={'line':'paragraphTextModern', 'original':'paragraphTextOriginal'})
refs = df[df.scripture==1].drop(columns=['scripture',]).rename(columns={'line':'paragraphRef'})
s = text.merge(refs, how='left', on=['chapter', 'paragraph', 'referenceNum'])
s['paragraphRef'] = s.paragraphRef.fillna("")
s['refArray'] = s.paragraphRef.apply(lambda x: [ref.strip() for ref in x.split(';') ])
s = s.drop(columns=['paragraphRef'])
s = s.rename(columns={'refArray':'paragraphRef'})
s = s.reset_index(drop=True)
s['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(s.shape[0])]
s['_type'] = 'confession'
s = s[['_id', '_type', 'chapter', 'paragraph', 'referenceNum', 'paragraphTextModern', 'paragraphTextOriginal','paragraphRef',]]

In [211]:
s

,_id,_type,chapter,paragraph,referenceNum,paragraphTextModern,paragraphTextOriginal,paragraphRef
0,1141bbc2-b85f-11ed-bec8-00000000b8de,confession,1,1,1,"The Holy Scriptures are the only sufficient, c...","The Holy Scripture is the only sufficient, cer...","[2 Timothy 3:15–17, Isaiah 8:20, Luke 16:29, 3..."
1,1141bc62-b85f-11ed-81c6-00000000acc3,confession,1,1,2,The light of nature and the works of creation ...,"Although the light of nature, and the works of...","[Romans 1:19–21, Romans 2:14,15, Psalm 19:1–3]"
2,1141bc94-b85f-11ed-8cb8-00000000756b,confession,1,1,3,"Therefore, the Lord was pleased at different t...",Therefore it pleased the Lord at sundry times ...,[Hebrews 1:1]
3,1141bcc6-b85f-11ed-ae27-00000000a675,confession,1,1,4,To preserve and propagate the truth better and...,and afterward for the better preserving and pr...,"[Proverbs 22:19–21, Romans 15:4, 2 Peter 1:19,..."
4,1141bcee-b85f-11ed-aae9-000000002220,confession,1,2,5,"The Holy Scriptures, or the Word of God writte...","Under the name of Holy Scripture, or the Word ...",[2 Timothy 3:16]
...,...,...,...,...,...,...,...,...
506,11420438-b85f-11ed-b53a-000000000be4,confession,32,2,6,"and punished with everlasting destruction, awa...","and punished with everlasting destruction, fro...","[Matthew 25:46, Mark 9:48, 2 Thessalonians 1:7..."
507,11420460-b85f-11ed-87fd-00000000b2eb,confession,32,3,7,Christ desires that we be firmly convinced tha...,As Christ would have us to be certainly persua...,"[2 Corinthians 5:10, 11]"
508,1142047e-b85f-11ed-8a00-0000000166d1,confession,32,3,8,and to comfort the godly more fully in their a...,and for the greater consolation of the godly i...,[2 Thessalonians 1:5–7]
509,114204a6-b85f-11ed-b278-00000000886e,confession,32,3,9,"For this reason, he has determined to keep the...","so will he have the day unknown to men, that t...","[Mark 13:35–37, Luke 12:35, 36]"


In [212]:
s.to_pickle('../data/1689_both_version_clean.pkl')

In [213]:
s.to_json("../cms/confession.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)